In [1]:
# Load Pandas into Python
import pandas as pd
from matplotlib import pyplot as plt
 
%matplotlib inline

In [6]:
# Read data
users = pd.read_csv('prices-split-adjusted.csv')

Examine for missing data
Narrow down list of stocks for analysis

In [7]:
users.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [14]:
users['symbol'].value_counts().head()

MTD     1762
TSN     1762
MMM     1762
MMC     1762
CELG    1762
Name: symbol, dtype: int64